<a href="https://colab.research.google.com/github/sudeepmenon17/PytorchforNN/blob/main/pth_to_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!pip install google-auth
from google.colab import auth
from google.colab import drive

!pip install tensorflow==2.12 keras==2.12

drive.mount('/content/drive')

# !pip install torchvision
!pip install onnx
# !pip install tensorflow
!pip install onnx_tf

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
import torch
import torch.nn as nn
import torch.onnx
from torch.autograd import Variable

# Define the PyTorch model class
class Model(nn.Module):
    def __init__(self, in_features=5, h1=61, h2=60, h3=40, out_features=1):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, h3)
        self.out = nn.Linear(h3, out_features)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        x = self.out(x)
        return x

In [46]:
# Load the saved PyTorch model
model = Model()
#model.load_state_dict(torch.load('/content/drive/MyDrive/komegaSST_data/airfoil_model.pt'))
model.load_state_dict(torch.load('/content/drive/MyDrive/model_final.pth'))
model.eval()

Model(
  (fc1): Linear(in_features=5, out_features=61, bias=True)
  (fc2): Linear(in_features=61, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=40, bias=True)
  (out): Linear(in_features=40, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [47]:
# Define file paths
onnx_file_path = "/content/drive/MyDrive/model_final.onnx"
dummy_input = torch.randn(1,5)

# Step 1: Export PyTorch model to ONNX format
def export_pytorch_to_onnx(model, dummy_input, onnx_file_path):
    torch.onnx.export(
        model,
        dummy_input,
        onnx_file_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output']
    )

In [48]:
# Perform the conversion
export_pytorch_to_onnx(model, dummy_input, onnx_file_path)


In [49]:
!pip install onnx2pytorch
import onnx
from onnx2pytorch import ConvertModel
import torch

# Load the ONNX model
onnx_model = onnx.load(onnx_file_path)

# Convert the ONNX model to a PyTorch model
pytorch_converted_model = ConvertModel(onnx_model)

# Save the converted PyTorch model state dictionary for later use if needed
torch.save(pytorch_converted_model.state_dict(), '/content/drive/MyDrive/converted_model_final.pth')

In [50]:
# Load the original PyTorch model and converted model for comparison
original_model = Model()
original_model.load_state_dict(torch.load('/content/drive/MyDrive/model_final.pth'))
original_model.eval()

converted_model = ConvertModel(onnx_model)
converted_model.load_state_dict(torch.load('/content/drive/MyDrive/converted_model_final.pth'))
converted_model.eval()

# Compare the weights
for (original_param, converted_param) in zip(original_model.parameters(), converted_model.parameters()):
    assert torch.allclose(original_param, converted_param, atol=1e-6), "The weights are different!"
print("All weights are correctly transferred.")

All weights are correctly transferred.


In [51]:
import numpy as np

# Generate a sample input tensor
input_data = np.random.randn(1, 5).astype(np.float32)
pytorch_input = torch.from_numpy(input_data)

# Get the original PyTorch model output
with torch.no_grad():
    original_output = original_model(pytorch_input).numpy()

# Get the converted PyTorch model output
with torch.no_grad():
    converted_output = converted_model(pytorch_input).numpy()

# Compare the outputs
print("Original PyTorch Model Output:", original_output)
print("Converted PyTorch Model Output:", converted_output)

# Check if the outputs are close
if np.allclose(original_output, converted_output, atol=1e-12):
    print("The outputs are the same within the tolerance level.")
else:
    print("The outputs are different.")

Original PyTorch Model Output: [[1.0292523]]
Converted PyTorch Model Output: [[1.0292523]]
The outputs are the same within the tolerance level.


In [52]:
import tensorflow as tf

# Define the TensorFlow model class
class KerasModel(tf.keras.Model):
    def __init__(self):
        super(KerasModel, self).__init__()
        self.fc1 = tf.keras.layers.Dense(61, activation='sigmoid')
        self.fc2 = tf.keras.layers.Dense(60, activation='sigmoid')
        self.fc3 = tf.keras.layers.Dense(40, activation='sigmoid')
        self.out = tf.keras.layers.Dense(1)

    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.out(x)
        return x

# Create an instance of the Keras model
keras_model = KerasModel()

# Trigger weight initialization with dummy input
dummy_input = tf.random.uniform((1, 5))  # batch size 1, input features = 5
_ = keras_model(dummy_input)

# Print layer names and weight shapes
for layer in keras_model.layers:
    print(f"{layer.name} - weights: {[w.shape for w in layer.get_weights()]}")

dense_12 - weights: [(5, 61), (61,)]
dense_13 - weights: [(61, 60), (60,)]
dense_14 - weights: [(60, 40), (40,)]
dense_15 - weights: [(40, 1), (1,)]


In [53]:
for layer in keras_model.layers:
    print(f"{layer.name} - weights: {[w.shape for w in layer.get_weights()]}")


dense_12 - weights: [(5, 61), (61,)]
dense_13 - weights: [(61, 60), (60,)]
dense_14 - weights: [(60, 40), (40,)]
dense_15 - weights: [(40, 1), (1,)]


In [54]:
import onnx

# Load ONNX model
onnx_model = onnx.load(onnx_file_path)

# Extract weights and biases from ONNX model
weights = {}
for initializer in onnx_model.graph.initializer:
    weights[initializer.name] = onnx.numpy_helper.to_array(initializer)

In [55]:
# Assign weights
keras_model.fc1.set_weights([weights['fc1.weight'].T, weights['fc1.bias']])
keras_model.fc2.set_weights([weights['fc2.weight'].T, weights['fc2.bias']])
keras_model.fc3.set_weights([weights['fc3.weight'].T, weights['fc3.bias']])
keras_model.out.set_weights([weights['out.weight'].T, weights['out.bias']])

# Dummy input to initialize
dummy_input = tf.random.normal([1, 5])
keras_model(dummy_input)

# ✅ Save in .tf format (SavedModel folder)
keras_model.save("/content/drive/MyDrive/tf_model_final", save_format="tf")

In [56]:
import numpy as np
import torch
import tensorflow as tf
from tensorflow import keras

# Sample input tensor
input_data = np.random.randn(1, 5).astype(np.float32)
pytorch_input = torch.from_numpy(input_data)

# Get PyTorch model output
with torch.no_grad():
    pytorch_output = model(pytorch_input).numpy()

# ✅ Load model via keras.models.load_model() to retain .predict()
final_tf_export_path = "/content/drive/MyDrive/tf_model_final"
tf_model = keras.models.load_model(final_tf_export_path)

# ✅ Run inference with .predict() (available now)
tf_output = tf_model.predict(input_data)

# Compare outputs
print("PyTorch Model Output:", pytorch_output)
print("TensorFlow Model Output:", tf_output)

if np.allclose(pytorch_output, tf_output, atol=1e-6):
    print("✅ The outputs are the same within the tolerance level.")
else:
    print("❌ The outputs are different.")



1/1 [==============================] - 0s 124ms/step
PyTorch Model Output: [[1.0663068]]
TensorFlow Model Output: [[1.0663072]]
✅ The outputs are the same within the tolerance level.
